In [343]:
%pip install GitPython

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\oheit\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [344]:
%pip install JPype1

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\oheit\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [345]:
from git import Repo
import pandas as pd
import numpy as np
import os
import fnmatch
from statistics import mean
import shutil
import sys
import subprocess
import logging

In [346]:
def getCommitsFrom(project):
    projectPath = "repositories/"+project
    repository = Repo(projectPath)
    try:
        repository.git.checkout("main", "-f")
    except:
        repository.git.checkout("master", "-f")
    iterCommits = repository.iter_commits()
    commits = list(iterCommits)
    print(f'{len(commits)} commits found')
    return commits

In [347]:
def checkoutTo(project, sha):
    projectPath = "repositories/" + project
    repository = Repo(projectPath)
    repository.git.checkout(sha, "-f")

In [348]:
def extractMetricsWithUnderstand(owner, project):
    if sys.platform == 'win32':
        understandPath = "understand/SciTools/bin/pc-win64/und"
    elif sys.platform == 'darwin':
        understandPath = "understand/SciTools/bin/mac/und"
    else:
        understandPath = "understand/SciTools/bin/linux/und"
        
    project_path = os.path.join("repositories", owner, project)
    
    if not os.path.exists('databases'):
        os.makedirs('databases')
        
    database_path = os.path.join('databases', project)
    metrics_output_file = f"{project}.csv"
    
    commands = [
        [understandPath, "create", "-languages", "java", database_path],
        [understandPath, "add", project_path, database_path],
        [understandPath, "settings", "-metrics", "all", database_path],
        [understandPath, "settings", "-metricsOutputFile", metrics_output_file, database_path],
        [understandPath, '-verbose', "analyze", '-all', project_path],
        [understandPath, "metrics", database_path]
    ]

    # Execute each command
    for command in commands:
        try:
            output = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        except subprocess.CalledProcessError as e:
            print(f"Error executing command {' '.join(command)}: {e}", file=sys.stderr)

    if os.path.exists(metrics_output_file):
        print(f"Metrics CSV file created at: {metrics_output_file}")
    else:
        print(f"No CSV file created. Please check the output above for errors.")

In [349]:
def method_level_metrics(dataframe, owner, project):
    dataframe = dataframe[dataframe["Kind"].str.contains("Method")]
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").mean()
    return dataframe

In [350]:
def class_level_metrics(dataframe, owner, project):
    dataframe = dataframe[dataframe["Kind"].str.contains("Class")]
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").mean()
    return dataframe

In [351]:
def project_level_metrics(dataframe, owner, project):
    numberJavaFiles = 0 if dataframe.empty else dataframe["Kind"].value_counts()["File"]
    dataframe = dataframe[dataframe["Kind"] == "Package"]
    packages = len(dataframe)
    dataframe.drop(dataframe.columns.difference(['CountDeclClass']), 1, inplace=True)
    dataframe["sample"] = owner+"/"+project
    dataframe = dataframe.groupby("sample").sum()
    dataframe["numberJavaFiles"] = numberJavaFiles
    dataframe["packages"] = packages
    return dataframe

In [352]:
def readMetricsFromCsv(project):
    dtype = {'Kind': str, 'Name': str, 'AvgCyclomatic': np.float64, 'AvgCyclomaticModified': np.float64,'AvgCyclomaticStrict': np.float64, 'AvgEssential': np.float64, 'AvgLine': np.float64, 'AvgLineBlank': np.float64,'AvgLineCode': np.float64, 'AvgLineComment': np.float64, 'CountClassBase': np.float64, 'CountClassCoupled': np.float64,'CountClassCoupledModified': np.float64, 'CountClassDerived': np.float64, 'CountDeclClass': np.float64,'CountDeclClassMethod': np.float64, 'CountDeclClassVariable': np.float64,'CountDeclExecutableUnit': np.float64, 'CountDeclFile': np.float64, 'CountDeclFunction': np.float64,'CountDeclInstanceMethod': np.float64, 'CountDeclInstanceVariable': np.float64,'CountDeclMethod': np.float64, 'CountDeclMethodAll': np.float64, 'CountDeclMethodDefault': np.float64,'CountDeclMethodPrivate': np.float64, 'CountDeclMethodProtected': np.float64,'CountDeclMethodPublic': np.float64, 'CountInput': np.float64,'CountLine': np.float64, 'CountLineBlank': np.float64,'CountLineCode': np.float64, 'CountLineCodeDecl': np.float64, 'CountLineCodeExe': np.float64,'CountLineComment': np.float64, 'CountOutput': np.float64, 'CountPath': np.float64, 'CountPathLog': np.float64,'CountSemicolon': np.float64, 'CountStmt': np.float64, 'CountStmtDecl': np.float64, 'CountStmtExe': np.float64,'Cyclomatic': np.float64, 'CyclomaticModified': np.float64, 'CyclomaticStrict': np.float64, 'Essential': np.float64, 'Knots': np.float64, 'MaxCyclomatic': np.float64, 'MaxCyclomaticModified': np.float64,'MaxCyclomaticStrict': np.float64, 'MaxEssential': np.float64,'MaxEssentialKnots': np.float64,'MaxInheritanceTree': np.float64, 'MaxNesting': np.float64, 'MinEssentialKnots': np.float64,'PercentLackOfCohesion': np.float64, 'PercentLackOfCohesionModified': np.float64,'RatioCommentToCode': np.float64, 'SumCyclomatic': np.float64, 'SumCyclomaticModified': np.float64,'SumCyclomaticStrict': np.float64, 'SumEssential': np.float64}
    dataframe = pd.read_csv(project+".csv", dtype=dtype)
    return dataframe

In [353]:
def getUnderstandMetrics(owner, project):
    extractMetricsWithUnderstand(owner, project)
    
    dataframe = readMetricsFromCsv(project)
    
    method_level = method_level_metrics(dataframe.copy(), owner, project)
    method_level["level"] = "method"
    method_level.set_index("level")

    
    class_level = class_level_metrics(dataframe.copy(), owner, project)
    class_level["level"] = "class"
    class_level.set_index("level")    
    
    project_level = project_level_metrics(dataframe.copy(), owner, project)
    project_level["level"] = "project"
    project_level.set_index("level")
    
    return pd.concat([method_level, class_level, project_level], axis=0)

In [354]:
def findPaths(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        if '.git' in root:
            continue
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [355]:
def deleteUnusedFiles(sample):
    os.remove(sample + ".csv")
    shutil.rmtree(sample + ".und")

In [356]:
def createDirectoryIfNotExists(dirName):
    if not os.path.exists(dirName):
            os.makedirs(dirName)

In [357]:
def getMetrics(commit, owner, project):
    metrics = getUnderstandMetrics(owner, project)
    metrics["commitSha"] = commit.hexsha
    metrics["commitDate"] = commit.authored_datetime
    return metrics

In [358]:
def extractMetricsByCommit(sample, commit):
    owner, project = sample.split("\\")
    checkoutTo(sample, commit.hexsha)
    print("commit ======= " + commit.hexsha)
    metrics = getMetrics(commit, owner, project)
    return metrics

In [359]:
def extractMetricsForAllCommits(commits, sample):
    allCommits = pd.DataFrame()
    for index, commit in enumerate(commits):
        metrics = extractMetricsByCommit(sample, commit)
        allCommits = allCommits.append(metrics, ignore_index=True)
        print("{0}% of commits completed from sample {1}".format((index/len(commits) * 100), sample))
        allCommits.to_csv("metrics\\"+sample+".csv", index=False)
    return allCommits

In [360]:
exclude = []
def metricsByCommit(samples):
    
    for index, sample in enumerate(samples):
        if (sample in exclude):
            continue
        print(sample)
        sample = sample.replace("/", "\\") #replace Sample Path For Windows Like
        
        print(f'{(index + 1 / len(samples)):.2%} samples completed')
        
        createDirectoryIfNotExists("metrics")
        
        commits = getCommitsFrom(sample)[::-1]
        
        owner, project = sample.split("\\")
        
        createDirectoryIfNotExists("metrics\\"+owner)
        
        allCommits = extractMetricsForAllCommits(commits, sample)
        
        deleteUnusedFiles(project)
        
        allCommits.to_csv("metrics\\"+sample+".csv", index=False)

In [361]:
samples = pd.read_csv(os.path.join('..', '1-GetQuestions', 'questions.csv'))

In [362]:
samples["full_name"] = samples["framework"] + "/" + samples["path"]

In [363]:
metricsByCommit(samples["full_name"].unique())

googlesamples/io2014-codelabs
0.54% samples completed
15 commits found
commit ======= 93d363b4f72eb520dd80520ed6ce331ead154a4d


Error executing command understand/SciTools/bin/pc-win64/und -verbose analyze -all repositories\googlesamples\io2014-codelabs: Command '['understand/SciTools/bin/pc-win64/und', '-verbose', 'analyze', '-all', 'repositories\\googlesamples\\io2014-codelabs']' returned non-zero exit status 2.
Error executing command understand/SciTools/bin/pc-win64/und metrics databases\io2014-codelabs: Command '['understand/SciTools/bin/pc-win64/und', 'metrics', 'databases\\io2014-codelabs']' returned non-zero exit status 2.


No CSV file created. Please check the output above for errors.


FileNotFoundError: [Errno 2] No such file or directory: 'io2014-codelabs.csv'